In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_local

In [53]:
page = cv2.imread('images/doc_scan.jpg')

In [54]:
cv2.imshow('tab',page)
cv2.waitKey()
cv2.destroyAllWindows()

In [55]:
page = cv2.resize(page, None, fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)

In [57]:
cv2.imshow('tab',page)
cv2.waitKey()
cv2.destroyAllWindows()

In [58]:
gray = cv2.cvtColor(page,cv2.COLOR_BGR2GRAY)

In [59]:
cv2.imshow('tab',gray)
cv2.waitKey()
cv2.destroyAllWindows()

In [60]:
blur = cv2.GaussianBlur(gray,(5,5),0)
cv2.imshow('tab',blur)
cv2.waitKey()
cv2.destroyAllWindows()

In [61]:
edge = cv2.Canny(blur,75,200)
edge_1 = edge.copy()
cv2.imshow('tab',edge)
cv2.waitKey()
cv2.destroyAllWindows()

In [62]:
contours = cv2.findContours(edge_1,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cont = sorted(contours[1], key=cv2.contourArea, reverse=True)[:5]
len(cont[0])

84

In [63]:
for c in cont:
    peri = cv2.arcLength(c,True)
    app = cv2.approxPolyDP(c,peri*0.02,True)
    print(peri)
    print(cv2.contourArea(c))
    
    if len(app) == 4:
        screen = app
        break
        

190.65180146694183
1023.0
1082.2610937356949
998.5
164.7523066997528
966.5
145.4385985136032
819.5
136.8528118133545
771.0


In [64]:
cv2.drawContours(page,[screen],-1,(0,255,0),3)
cv2.imshow('tab',page)
cv2.waitKey()
cv2.destroyAllWindows()
        
    

In [65]:
p = screen.sum(axis=1)
np.zeros((4,2),dtype='float32')

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]], dtype=float32)

In [66]:
def order_pts(pts):
    rect = np.zeros((4,2),dtype='float32')
    
    s = pts.sum(axis=1)
    print(s)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)
    print(diff)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

In [67]:
def four_point_transform(image,pts):
    
    rect = order_pts(pts)
    (tl,tr,br,bl) = rect
    print(tl)
    print(tr)
    print(br)
    print(bl)
    
    widthA = np.sqrt(((br[0]-bl[0])**2)+((br[1]-bl[1])**2))
    widthB = np.sqrt(((tr[0]-tl[0])**2)+((tr[1]-tl[1])**2))
    maxWidth = max(int(widthA),int(widthB))
    print(widthA)
    print(widthB)
    heightA = np.sqrt(((tr[0]-br[0])**2)+((tr[1]-br[1])**2))
    heightB = np.sqrt(((tl[0]-bl[0])**2)+((tl[1]-bl[1])**2))
    print(heightA)
    print(heightB)
    maxHeight = max(int(heightA),int(heightB))
    
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0,maxHeight-1]],dtype='float32')
    
    M = cv2.getPerspectiveTransform(rect,dst)
    warped = cv2.warpPerspective(image,M,(maxWidth,maxHeight))
    return warped

In [68]:
warped = four_point_transform(page,screen.reshape(4,2))

warped = cv2.cvtColor(warped,cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset=10,method='gaussian')
warped = (warped>T).astype('uint8')*255

[ 182  845 1283  653]
[[  -4]
 [ 645]
 [ 103]
 [-421]]
[ 93.  89.]
[ 537.  116.]
[ 590.  693.]
[ 100.  745.]
492.751458648
444.820188391
579.429029304
656.037346498


In [69]:
cv2.imshow('tab',warped)
cv2.waitKey()
cv2.destroyAllWindows()